In [9]:
import pandas as pd
from konlpy.tag import Twitter

In [10]:
naver_user = pd.read_csv('[Final]naver_user_table.csv',sep='\t',encoding='utf-8')
dbdbdeep_user = pd.read_csv('[Final]dbdbdeep_user_table.csv',sep='\t',encoding='utf-8')

In [11]:
def refined_category(category):
    refined_list = []
    for i in category.split('], '):
        try:
            category_count = int(i.split(', ')[0].replace('[','').replace("'",''))
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((category_count,categoty_title))
        except:
            pass
    return refined_list

In [12]:
def refined_category_title(category):
    refined_list = []
    for i in category.split('], '):
        try:
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((categoty_title))
        except:
            pass
    return refined_list

In [13]:
korean_stopword = pd.read_csv('Korean_stopwords.txt',encoding='utf-8',header=None)
korean_stopword_list = korean_stopword[0].tolist()
stopwords = ['하다', ',', '들', '이', '..', '.', '것', '다', '이다', '~', '그', '그녀', '저', '...', '"', '~~',"'",':','&','[',']','{','}','(',')','-'] 
sum_stopwords = stopwords + korean_stopword_list

In [14]:
def Korean_tagger(Text):

    twitter = Twitter()

    try:
        malist = twitter.pos(Text, norm=True, stem=True)
        r = []
        remove_tag =['Eomi','PreEomi','Josa','Determiner','Foreign','Alpha','Number','Punctuation','Suffix','Unknown','Hashtag','KoreanParticle','ScreenName']

        stopwords = sum_stopwords


        for word in malist:
            # 어미/조사/구두점/ㅋㅋ^^ㅎㅎ/음표살림/Alphabet/부사는 대상에서 제외
            if not word[1] in remove_tag:
                if not word[0] in r:
                    if not word[0] in stopwords:
                       # 숫자, 특수문자 제거.
                        r.append(word[0])
        return ' '.join(r)
    except Exception as e:
        return ''

In [15]:
naver_user['Blog_info_text_embedding'] = naver_user['Blog_info_text'].apply(lambda x : Korean_tagger(x))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
dbdbdeep_user['Blog_info_text_embedding'] = dbdbdeep_user['Blog_info_text'].apply(lambda x : Korean_tagger(x))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [17]:
Total_user = pd.concat([naver_user,dbdbdeep_user])

In [18]:
Total_test = Total_user[['Blog_info_text_embedding','Credibility']]

In [19]:
Info_text = Total_test['Blog_info_text_embedding'].tolist()

## Tfidf

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=161)
x = v.fit_transform(Info_text)

In [21]:
y = Total_test['Credibility'].tolist()

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.712450592885


In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

          0       0.73      0.90      0.81       672
          1       0.64      0.34      0.44       340

avg / total       0.70      0.71      0.68      1012



In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(result, x, y, cv=5)
scores                                              

array([ 0.71498371,  0.72638436,  0.67536705,  0.68627451,  0.74836601])

## Doc2vec

In [26]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [43]:
Total_test = Total_user[['Blog_info_text_embedding','Credibility']]

In [44]:
text = Total_test['Blog_info_text_embedding'].tolist()

In [45]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [46]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [47]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [48]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=92,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [49]:
doc_vectorizer.build_vocab(sentences)

2018-09-02 11:35:20,275 : INFO : collecting all words and their counts
2018-09-02 11:35:20,277 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-02 11:35:20,278 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-02 11:35:20,315 : INFO : collected 1076 word types and 3065 unique tags from a corpus of 3065 examples and 70462 words
2018-09-02 11:35:20,316 : INFO : Loading a fresh vocabulary
2018-09-02 11:35:20,319 : INFO : min_count=20 retains 377 unique words (35% of original 1076, drops 699)
2018-09-02 11:35:20,320 : INFO : min_count=20 leaves 66888 word corpus (94% of original 70462, drops 3574)
2018-09-02 11:35:20,321 : INFO : deleting the raw counts dictionary of 1076 items
2018-09-02 11:35:20,326 : INFO : sample=0.001 downsamples 67 most-common words
2018-09-02 11:35:20,328 : INFO : downsampling leaves estimated 36255 word corpus (54.2% of prior 66888)
2018-09-

In [50]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d92,n10,hs,w8,mc20,s0.001,t4)


In [51]:
doc_vectorizer.corpus_count*0.03

91.95

In [52]:
doc_vectorizer.iter

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [53]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-02 11:35:24,516 : INFO : training model with 4 workers on 377 vocabulary and 92 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-02 11:35:24,712 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:35:24,838 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:35:24,844 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:35:24,846 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:35:24,847 : INFO : EPOCH - 1 : training on 70462 raw words (39317 effective words) took 0.3s, 121884 effective words/s
2018-09-02 11:35:25,044 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:35:25,160 : IN

2018-09-02 11:35:29,426 : INFO : EPOCH - 5 : training on 70462 raw words (39415 effective words) took 0.3s, 126458 effective words/s
2018-09-02 11:35:29,426 : INFO : training on a 352310 raw words (196808 effective words) took 1.6s, 120659 effective words/s
2018-09-02 11:35:29,427 : INFO : training model with 4 workers on 377 vocabulary and 92 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-02 11:35:29,634 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:35:29,748 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:35:29,751 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:35:29,755 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:35:29,756 : INFO : EPOCH - 1 : training on 70462 raw words (39279 effective words) took 0.3s, 123430 effective words/s
2018-09-02 11:35:29,946 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-0

2018-09-02 11:35:34,302 : INFO : EPOCH - 5 : training on 70462 raw words (39333 effective words) took 0.3s, 120455 effective words/s
2018-09-02 11:35:34,303 : INFO : training on a 352310 raw words (196681 effective words) took 1.6s, 120917 effective words/s
2018-09-02 11:35:34,304 : INFO : training model with 4 workers on 377 vocabulary and 92 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-02 11:35:34,501 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:35:34,614 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:35:34,623 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:35:34,629 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:35:34,630 : INFO : EPOCH - 1 : training on 70462 raw words (39355 effective words) took 0.3s, 124823 effective words/s
2018-09-02 11:35:34,826 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-0

2018-09-02 11:35:39,464 : INFO : EPOCH - 5 : training on 70462 raw words (39328 effective words) took 0.3s, 125138 effective words/s
2018-09-02 11:35:39,465 : INFO : training on a 352310 raw words (196709 effective words) took 1.7s, 113522 effective words/s
2018-09-02 11:35:39,466 : INFO : training model with 4 workers on 377 vocabulary and 92 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-02 11:35:39,669 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:35:39,784 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:35:39,794 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:35:39,803 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:35:39,804 : INFO : EPOCH - 1 : training on 70462 raw words (39291 effective words) took 0.3s, 120512 effective words/s
2018-09-02 11:35:39,996 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-0

During Time: 16.624151945114136


In [54]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [55]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

## SVM

In [56]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.728260869565


# MLP

In [57]:
from sklearn.neural_network import MLPClassifier

In [58]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [60]:
start = time.time()
mlp_clf.fit(X_train, y)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.63657523
Iteration 2, loss = 0.57108310
Iteration 3, loss = 0.53959295
Iteration 4, loss = 0.52346447
Iteration 5, loss = 0.51377926
Iteration 6, loss = 0.50754608
Iteration 7, loss = 0.49968620
Iteration 8, loss = 0.49422984
Iteration 9, loss = 0.48718201
Iteration 10, loss = 0.48103360
Time: 0.112207s


/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [64]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.771
